In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import numpy as np
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import os
import copy



Matplotlib is building the font cache; this may take a moment.


In [4]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
mean=np.array([0.485,.425,.435])

std=np.array([0.229,0.245,0.225])


In [13]:
data_transforms={'train':transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),transforms.Normalize(mean,std)]),
                'val':transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),
                                            transforms.ToTensor(),transforms.Normalize(mean,std)])}

In [24]:
data_dir='/home/umanagendramalla/Downloads/hymenoptera_data'

In [15]:
sets=['train','val']

In [25]:
image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}

In [39]:
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=0) for x in ['train','val']}

In [41]:
dataset_sizes={x:len(data_loaders[x]) for x in ['train','val']}
class_names=image_datasets['train'].classes

In [28]:
print(class_names)

['ants', 'bees']


In [44]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=25):
    since=time.time()
    best_model_weights=copy.deepcopy(model.state_dict())
    best_acc=0.0
    for epoch in range(num_epochs):
        print(f'epoch{epoch}/{num_epochs-1}')
        print('-'*10)
        for phase in ['train','val']:
            if(phase=='train'):
                model.train()
            else:
                model.eval()
            running_loss=0.0
            running_corrects=0.0
            for inputs,labels in dataloaders[phase]:
                inputs=inputs.to(device)
                labels=labels.to(device)
                with torch.set_grad_enabled(phase=='train'):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs,1)
                    loss=criterion(outputs,labels)
                    if(phase=='train'):
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()    
                running_loss+=loss.item()*inputs.size(0)
                running_corrects+=torch.sum(preds==labels.data)
            if(phase=='train'):
                scheduler.step()
            epoch_loss=running_loss/dataset_sizes[phase]
            epoch_acc=running_corrects.double()/dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
                

In [31]:
model=models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/umanagendramalla/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100.0%


In [34]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)


In [35]:
criterion = nn.CrossEntropyLoss()


In [36]:
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [37]:
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [46]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)


epoch0/1
----------
train Loss: 1.4613 Acc: 3.3770
val Loss: 0.7892 Acc: 3.7436

epoch1/1
----------
train Loss: 1.5085 Acc: 3.2295
val Loss: 0.7669 Acc: 3.7436

Training complete in 5m 3s
Best val Acc: 3.743590


In [47]:
models=models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad=False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)



epoch0/1
----------
train Loss: 2.6431 Acc: 2.2951
val Loss: 2.0523 Acc: 2.8462

epoch1/1
----------
train Loss: 2.1999 Acc: 2.9344
val Loss: 1.4004 Acc: 3.5641

Training complete in 1m 38s
Best val Acc: 3.564103
